In [9]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import xgboost as xgb

In [12]:
data = pd.read_csv("Salinan dari TAGS v6.1.9.1 - Archive (34).csv", sep=',', encoding='utf=8')
data.head()

,Tweets,VALUE
0,sampe lupa cara bikin application letter dong....,Negatif
1,gw ngakakin ini sampe bego https://t.co/xP3o5o...,Negatif
2,@dostoyevzky apseh bg oloap :v,Positif
3,taro singing any song in the bg ma he's adorable,Positif
4,@NagleKara @yeetwheat99 @QEDmostly @BillKristo...,Positif


In [13]:
# proses case folding
import re 
def casefolding(Tweets):
    Tweets = Tweets.lower()
    Tweets = Tweets.strip(" ")
    Tweets = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', '', Tweets)
    return Tweets 
data['Tweets'] = data['Tweets'].apply(casefolding)
data.head(1000)

,Tweets,VALUE
0,sampe lupa cara bikin application letter dong ...,Negatif
1,gw ngakakin ini sampe bego,Negatif
2,apseh bg oloap v,Positif
3,taro singing any song in the bg ma hes adorable,Positif
4,this marine blocked me lmfao,Positif
...,...,...
994,aku juga bisa koq aku doakan semoga yahya wal...,Negatif
995,kalo lucu sih aku ora seng anjing sangarr yo ...,Positif
996,sarah 1st timer di nasi lemak anjing vs tangan...,Positif
997,bisaa lo bayangin gw ada kucing sama anjing d...,Positif


In [14]:
# proses tokenizing 
def token(Tweets):
    nstr = Tweets.split(' ')
    dat = []
    a = -1 
    for hu in nstr:
        a = a + 1 
    if hu == '': 
        dat.append(a)
    p = 0 
    b = 0
    for q in dat:
        b = q - p 
        del nstr [b]
        p = p + 1 
    return nstr 
data['Tweets'] = data['Tweets'].apply(token)
data.head(1000)

,Tweets,VALUE
0,"[sampe, lupa, cara, bikin, application, letter...",Negatif
1,"[gw, ngakakin, ini, sampe, bego]",Negatif
2,"[, apseh, bg, oloap, v]",Positif
3,"[taro, singing, any, song, in, the, bg, ma, he...",Positif
4,"[, , , , this, marine, blocked, me, lmfao]",Positif
...,...,...
994,"[, aku, juga, bisa, koq, aku, doakan, semoga, ...",Negatif
995,"[, kalo, lucu, sih, aku, ora, seng, anjing, sa...",Positif
996,"[sarah, 1st, timer, di, nasi, lemak, anjing, v...",Positif
997,"[, bisaa, lo, bayangin, gw, ada, kucing, sama,...",Positif


In [15]:
# proses filtering 
import nltk 
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 

def stopword_removal(Tweets): 
    filtering = stopwords.words('indonesian','english')
    x = []
    data = []
    def myFunc(x): 
        if x in filtering: 
            return False 
        else:
            return True 
    fit = filter(myFunc, Tweets)
    for x in fit: 
        data.append(x)
    return data
data['Tweets'] = data['Tweets'].apply(stopword_removal)
data.head(1000)

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>


,Tweets,VALUE
0,"[sampe, lupa, bikin, application, letter, sump...",Negatif
1,"[gw, ngakakin, sampe, bego]",Negatif
2,"[, apseh, bg, oloap, v]",Positif
3,"[taro, singing, any, song, in, the, bg, ma, he...",Positif
4,"[, , , , this, marine, blocked, me, lmfao]",Positif
...,...,...
994,"[, koq, doakan, semoga, yahya, waloni, mati, d...",Negatif
995,"[, kalo, lucu, sih, ora, seng, anjing, sangarr...",Positif
996,"[sarah, 1st, timer, nasi, lemak, anjing, vs, t...",Positif
997,"[, bisaa, lo, bayangin, gw, kucing, anjing, de...",Positif


In [16]:
# proses stemming

from sklearn.pipeline import Pipeline 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory 

def stemming(Tweets):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in Tweets:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean=[]
    d_clean= " ".join(do) 
    print(d_clean)
    return d_clean 
data['Tweets'] = data['Tweets'].apply(stemming)

data.to_csv('data_clean.csv', index=False)
data_clean = pd.read_csv('data_clean.csv', encoding='latin1')
data_clean.head()

sampe lupa bikin application letter sumpah bego bgt aing
gw ngakakin sampe bego
 apseh bg oloap v
taro singing any song in the bg ma hes adorable
    this marine blocked me lmfao
photo at cafe credit bg  cafe cafe digitalart artoftheday art artwork digitalpainting digital
  ya kalo tegak hukum mah beda bg salah kl diem aja konteks nya dosa yg ga langgar hukum ga ngerugiin orang jg
you are so beautiful the bg just blurs around you
 babi monyet
 enak babi no works no school no stressed just eat sleep and ngik ngok ngik ngok
suka diam jela babi tayah pekak
udah liat bngt si anak babi  iri hati ckck
dont get me wrong never kong hiningi sa kanya ang salary niya cs i have my own and mas malak ako kumita sa kanya but maliit malak andun yung siya na babi may pera na sa card ko pag kailangan mo andun lang kunin mo nalang as in i am just lucky to have him hahaha
dh ramai duk complain pasal service kau yg mcm babi nak tnya adu pundekkk 
  nafkah rela jd babi
 babi anjing
 babi monyet
tahula masak

 honestly what did i just read rexs wordsalads are becoming more incoherent by the day where is the article are all these facts located  more headlines created to fck with the rubes retirerex
 fck you pepnot giving a chance to kdb
fck i keep forgetting i dont have facebook
anjir iya rosaria mantep
 liat lucu anjir ki wkwkwk ga bayang kalo tangan gitu jg
beli skincare 200k mahal banget anjirbeli kertas ganteng 200k anjir murah banget langsung co langsung tf
ayo tumbuk anjir
 gue receh bgt anjir baca replyan lo ketawa
knp dekat puasa gue nolep anjir
 anjir
anjir
  anjir flaa gue paham
  pantesan anjir dicepuin  boro ek bebeja isu
 terbang anjir
 anjir getol wkwk
anjir kena unfoll thaifess garagara on dah
sepi ak yg flop si anjir dahla sedi bgt close bai
  eh anjir lu udh gw kasih undang bom si
 iya ak emank gemesin anjir cuk km gtw ak pcar mu
gua join space nya ren gada suara anjir
kesel banget anjir kondisi batuk batuk tau gmn
ati karo logika rak sinkron anjir
  anjir salah rep tpi hasi

 fuck my balls fucking hurt
we need to open the fuck up
and fuck billboardtwiceits time to turn the notifications off
 bus    sally   never said they werent but i still dont give a fuck
sterling dey fuck up
 fuck sake i hate people
 heavily gut bumpsfuck yeah
 fuck i cant wait
nicholas jesus fukin h christ i think i will build a wailing wall so all these brainwashed sheeple can get together and mourn theirselves tae fuck
 fuck it im going to do plan b
oh fuck x2
tsimikas is as pointless as davies when klopp is still around man fuck this club and fuck that man
i salute the girls who do squats everyday because fuck thattttt
  what the fuck i was like did i even comment that
ang swerte ko fuck
ik this is cheesy and annoying and you prolly dont even give a fuck but i love my bf and he cute as hell
woke up to my ex and my ex best friend having blocked me last night and now im confused as to what the fuck happened and why they both did it at the same time
 holy fuck meron god its been so lon

        but thats the thing if the op was dressed up with another man he wouldnt be calling it no friendship date he would say me and my boy going to eat or some bullshit he called it a date because its an attractive female if they were just friends why it had to be a date
  thats bullshit
 no bullshit im done tron understand it
 anjing virtual kak
ngapain kuliah2 kayak lulus ajagua sumpahin kuliah lu ga lulusngapain ngurusin kuliah lo gabakal lulus itusemua kayak gin gara2 lo kuliah2 anjingeh anjing klo w cakep w udah ngejablay dru bajing u smua
 ngga anjing jugaaa
ih bagus anjing kentut
 cicak kek anjing cicak
 tu filter ajg smg km eg kya ajg y
 things gotta change in the law enforcement because this crackers be thinking that because they say jump we got to jump and no fuck that things have changed
what in the fuck livavl lfc
 2 burgers 2 chips and a drink for 2 how the fuck did them niggas profit off of that khalil
youre beautiful okay  fuck societys standards
fuck keyboard
fuck off

 anjing gk derita ny te sih hidup ny lacar aja gak adil bangett au jd belok te hana yah
anjing
 anjing brutau
anjing
anjing
suara anjing gong gong jdi inget bella livenya dejun kemaren wkwk
  dream 127 anjing translate in inggris
ih anjing anjing
anjing
hahaha mihoyo anjing depelopernya ga pinter bikin game hahahabismillah zhongli one pull
explore ig lg postingan anjing corgi pengeeeennnn
 jih anjing kak
 mjbwoyyy elah gue baper banget anjing sjlsjsksks fvckkkk pls sweet banget aja gue udah kaya anjrid bnr bnr u lucky bgt pls
kambing kucing monyet beruang burung babi anjing
 gajelas bgt anjing sok tau
 gaya anjing
w bus mana sih anjingkok w pusing sendiriturun mana w tar
anjing ak malu diomongin grup hduh ppek
mak gua nambah anjing yallahh
  anjing iya ga bawa gmn gue rumah loo kah
 sembarang aja si anjing ngomongnya 
mmm anjing
 anjing ngapain gue mas waras
biy chat adin capslock dmn anjing dmn untung sayang
anjing
 tau anjing ga
 anjing
 anjing
 buru anjing malam hahahaha
 anjing
cun

 ga dah kalo anjing
  crita huntu kemaren anjing kosan gonggong pagi2 hiii seremm
 twt anjing vote dynamite for bestmusicvideo  at 2021 iheartawards twt
 anjing sie
anjing
 anjing
 anak anjing
w gin anjing 
air dah tidur nk nafas takut tp tulah tu org sibuk duk tukang kat bawak kereta start enjin anjing salak kapal terbang org jerit
 yahh anjing lelah
 twt anjing serigala harimau aneh ga si lucu i vote for dynamite for bestmusicvideo at the 2021 iheartawards twt
anjing
patung ya anjing
kejar anjing yg lat heheman superherokocak
ketemu anjing lari wkwk
 syounn anjing gua nama oje
  blinkers anjing
result togel bus  64  8140 sahshio   anjingunsur  logamlink shiokambing 1link shiokambing 2link shiokambing 3link shiokambing 5
anjing
gua sedih bgt anjing winwin orang islam ya
ocap ga bolehin renang kepala nyelem   jd ken gaya renang kaya anjing yg kepala atas skay suka latih bareng tp ga bisa2 alhasil tenggelem hahaha  coba bisain mgkn dibolehin skrg 3
 anjing ngapa 2 qrt gaib
kalo ultimate

 twt anjing haram i vote dynamite for bestmusicvideo at the 2021 iheartawards twt
 bias emot kelinci kucing anjing
 slahh anjing gw gmna sih
 1 anjing
anjing
  anjing maksutnya suswu
ni bangun tidur sadar trus hunt lord kayak bangun trus lari kejar anjing
 anak anjing
  binatang anjing
awkarin mff anjing
  ritual tp suka pull anjing jembatan liyue emg fav spot genshin trs suka main anjing
ken banget anjing kaya yeontan  gemes banget  yg
 anjing
anjing kayak akun alter
 anjing
buka norak gua liat h2 anjing keren
suka kesel anak kakek lu udah sakit operasi perhati lu ga ngentot  lu gampangin kasih uang ada ehh anjing gw gw bolak rs liat kakek sakit emang ga capek ngentot  anak ga tau bajing
 pawang anjing
sumpah klo pelihara ken anjing kaya gin gue 
  makan daging anjing sayur kol
lucu bgt dah orang main gartic trus suruh gambar anjing gambar tulis mantan
nas  kena kejar anjing precut
seokyung tuh cantik bgt anjing ga pacar aing
lu gue nyerah engga anjing gue diem diem aja stalking lo
 s

,Tweets,VALUE
0,sampe lupa bikin application letter sumpah beg...,Negatif
1,gw ngakakin sampe bego,Negatif
2,apseh bg oloap v,Positif
3,taro singing any song in the bg ma hes adorable,Positif
4,this marine blocked me lmfao,Positif


In [17]:
data_clean = data_clean.astype({'VALUE' : 'category'})
data_clean = data_clean.astype({'Tweets' : 'object'})
data_clean.dtypes

Tweets      object
VALUE     category
dtype: object

In [20]:
# proses TF-IDF dan 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 

tvec = CountVectorizer()
X_cVec = tvec.fit_transform(data_clean['Tweets'].values.astype('U'))
print(X_cVec)
h_tfidf = TfidfTransformer()
x_tfidf = h_tfidf.fit_transform(X_cVec)
print(x_tfidf)
X = data_clean.Tweets
Y = data_clean.VALUE

  (0, 2624)	1
  (0, 1858)	1
  (0, 445)	1
  (0, 242)	1
  (0, 1791)	1
  (0, 2898)	1
  (0, 391)	1
  (0, 431)	1
  (0, 113)	1
  (1, 2624)	1
  (1, 391)	1
  (1, 1246)	1
  (1, 2114)	1
  (2, 246)	1
  (2, 427)	1
  (2, 2246)	1
  (3, 427)	1
  (3, 2949)	1
  (3, 2772)	1
  (3, 223)	1
  (3, 2834)	1
  (3, 1421)	1
  (3, 3008)	1
  (3, 1859)	1
  (3, 1327)	1
  :	:
  (995, 2703)	1
  (995, 2630)	1
  (995, 3250)	1
  (996, 195)	1
  (996, 2943)	1
  (996, 19)	1
  (996, 2081)	1
  (996, 2635)	1
  (996, 3042)	1
  (996, 1781)	1
  (996, 3206)	1
  (996, 1936)	1
  (997, 1246)	1
  (997, 195)	1
  (997, 1825)	1
  (997, 1200)	1
  (997, 1722)	1
  (997, 1643)	1
  (997, 363)	1
  (997, 457)	1
  (997, 370)	1
  (997, 809)	1
  (998, 431)	1
  (998, 195)	1
  (998, 560)	1
  (0, 2898)	0.3165358211515389
  (0, 2624)	0.2879185564894961
  (0, 1858)	0.32472023570052744
  (0, 1791)	0.3830495172135814
  (0, 445)	0.3165358211515389
  (0, 431)	0.23752845306128
  (0, 391)	0.3615219149115587
  (0, 242)	0.3830495172135814
  (0, 113)	0.361521914

In [21]:
# splitting data 
from sklearn.model_selection import train_test_split 

from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

kf = KFold(n_splits=10)
X_array = x_tfidf.toarray()
def cross_val(estimator):
    acc = []
    pcs = []
    rec = []
    
    for train_index, test_index in kf.split(X_array, Y):
        X_train, X_test = X_array[train_index], X_array[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        model = estimator.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        acc.append(accuracy_score(y_test, y_pred))
        pcs.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        rec.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        rec.append(f1_score(y_test, y_pred, average='macro', zero_division=0))
        
        print(classification_report(y_test, y_pred, zero_division=0))
        print(f'confusion matrix:\n {confusion_matrix(y_test, y_pred)}')
        print('================================================\n')
    
    print(f'average Akurasi: {np.mean(acc)}')
    print(f'average Presisi: {np.mean(pcs)}')
    print(f'average recall: {np.mean(rec)}')
    print(f'average f1-score {np.mean(rec)}')

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
cross_val(nb)

              precision    recall  f1-score   support

     Negatif       0.91      0.54      0.68        89
     Positif       0.13      0.55      0.21        11

    accuracy                           0.54       100
   macro avg       0.52      0.54      0.44       100
weighted avg       0.82      0.54      0.62       100

confusion matrix:
 [[48 41]
 [ 5  6]]

              precision    recall  f1-score   support

     Negatif       1.00      0.54      0.70        98
     Positif       0.04      1.00      0.08         2

    accuracy                           0.55       100
   macro avg       0.52      0.77      0.39       100
weighted avg       0.98      0.55      0.69       100

confusion matrix:
 [[53 45]
 [ 0  2]]

              precision    recall  f1-score   support

     Negatif       0.98      0.96      0.97        98
     Positif       0.00      0.00      0.00         2

    accuracy                           0.94       100
   macro avg       0.49      0.48      0.48       